# What is Functional Programming?
* From HaskellWiki: *"Functional programming is a style of programming which models computations as the evaluation of expressions."*
* Functional Programming is programming without state
  * No mutability!
  * Referential Transparency!
  * Great for pipelines!
* Another view: Functional programs describe what things are instead of how to do them

In [1]:
fact 0 = 1
fact n = n * fact (n-1)
-- Pro way of doing this: fact n = product [1..n]

fact 10

3628800

# Benefits of Functional Programming, or: why do you care?
* Clean, modular code
* Easy to refactor
* Easy to make abstractions
* Allows for *tons* of aggressive compliler optimizations
* Can look at functions in isolation
* Often trivially parallizeable
  * No state -> no data races
  * No state -> extremely lightweight threads

In [2]:
import Control.Parallel.Strategies (parMap, rseq) -- from `parallel` package

-- Non-parallel version
incrementAll xs = map (+1) xs

-- Parallel version
incrementAll' xs = parMap rseq (+1) xs

incrementAll [0,1,2,3]
incrementAll' [0,1,2,3]

[1,2,3,4]

[1,2,3,4]

# Examples of Functional Languages
**Lisp Family**
* Common Lisp
* Clojure
* Scheme
* Racket

**ML Family**
* Haskell <- This is what we will be using today
* OCaml
* Erlang
* Scala
* Elm

# You are already using functional programming!
* Lambdas
* Recursion
* Arguably generators!
* So take the leap! Cast off your imperative chains!

# Hello, Haskell

In [3]:
-- "Hello, World!" in Haskell
main = putStrLn "Hello, World!"

:t main
:t putStrLn
:t "Hello, World!"
main

main :: IO ()

putStrLn :: String -> IO ()

"Hello, World!" :: [Char]

Hello, World!

## Map Filter Reduce
* General approach for solving large classes of problems
* Lends itself well to parallelization, distributed computing, etc
* Strategy:
  * Start with a large dataset
  * Apply some transformation to each (`map`)
  * Cut down to just the interesting results (`filter`)
  * Summarize (`reduce`)
* `map`, `filter`, and `reduce` are the name of the python functions. In Haskell, they are `map`, `filter`, and `foldl`/`foldr`.

In [4]:
:t map
map negate [0..10]

map :: forall a b. (a -> b) -> [a] -> [b]

[0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10]

In [5]:
:t filter
filter even [0..10]

filter :: forall a. (a -> Bool) -> [a] -> [a]

[0,2,4,6,8,10]

In [6]:
foldl (+) 0 [1..10]

Line 1: Use sum
Found:
foldl (+) 0
Why not:
sum

55

In [7]:
import Data.Char (digitToInt)

readInt :: String -> Int
readInt to_read = foldl step 0 to_read
    where step old new = old*10 + digitToInt new
    
readInt "123"

123

# Map Filter Reduce for Weak Lensing Pipeline
* Goal: Take a bunch of raw images, process them, and combine only the high quality ones

In [8]:
-- Weak lensing pipeline

process :: RawImage -> ProcessedImage
process = undefined

highQuality :: ProcessedImage -> Bool
highQuality = undefined

addToStack :: Stack -> ProcessedImage -> Stack
addToStack = undefined

emptyStack :: Stack
emptyStack = undefined

pipeline :: [RawImage] -> Stack
pipeline = foldl addToStack emptyStack . filter highQuality . map process

parallel_pipeline :: [RawImage] -> Stack
parallel_pipeline = foldl addToStack emptyStack . filter highQuality . parMap rseq process

data RawImage = RawImage {
    -- Raw image data here
    }

data ProcessedImage = ProcessedImage {
    -- Process image data here
    }
    
data Stack = Stack {
    -- Stacked image data here
    }

# Thinking Functionally

## Printing first 100 primes

In [9]:
primes :: [Integer]
primes = 2 : filter isPrime [3..]

isPrime :: Integer -> Bool
isPrime n = divisibleByNone n relevantPrimes
    where relevantPrimes = takeWhile relevant primes
          relevant p = p*p <= n

divisibleByNone :: Integer -> [Integer] -> Bool
divisibleByNone n = not . any (divides n)
    where divides n p = n `mod` p == 0

main :: IO ()
main = print . take 100 $ primes

main

-- Do < 1000 example

-- See primes.py for imperative comparison!

[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367,373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,461,463,467,479,487,491,499,503,509,521,523,541]

# But Devon! What about inherently state-y things?
## Implementing A Solar Simulation

In [10]:
data SolarState = SolarState {
        radius :: Double,
        metallicity :: Double
        -- ... other variables
    }

step :: SolarState -> SolarState
step = undefined

done :: SolarState -> Bool
done = undefined

runSim :: SolarState -> SolarState
runSim initial_state = until done step initial_state

doNsteps :: Int -> SolarState -> SolarState
doNsteps n initial_state = iterate step initial_state !! n

firstNSteps :: Int -> SolarState -> [SolarState]
firstNSteps n initial_state = take n $ iterate step initial_state

## Okay, but what about really, really inherently state-y things?
### RNGs
* Usually implemented as `nextVal :: Rng -> (a, Rng)`
* Lots of ways of hiding this, so it's less obnoxious to work with. Example: type `Rand g Int`.

### Input / Output
* Different functional languages deal with this differently
* In Haskell, all functions that interact with the rest of the world *must* be marked IO
  * Example: `readFile :: Filename -> IO String`
  * Usually these functions can be separated nicely from everything else!
  * If not, then it's good to know that your function is (arguably) not referentially transparent (hence the IO marker)

# Functional Error Handling

In [11]:
first :: [a] -> a
first (x:xs) = x

first [1,2,3,4,5]
first "Devon is cool"

-- Show them the error here

1

'D'

In [12]:
-- data Maybe a = Nothing | Just a

first' :: [a] -> Maybe a
first' (x:xs) = Just x
first' [] = Nothing

first' [1,2,3]
first' []

Just 1

Nothing

In [13]:
-- Using this, we can force ourselves to deal with edge cases!

-- If we use the bad version of first, we can get into trouble
first_incremented :: Num a => [a] -> a
first_incremented xs = first xs + 1
first_incremented [3,2,1]
-- first_incremented [] -- error

first_incremented' :: Num a => [a] -> Maybe a
first_incremented' xs =
    case first' xs of
        Just n -> Just (n + 1)
        Nothing -> Nothing

first_incremented' [3,2,1]
first_incremented' []

-- We can do this more cleanly as:
first_incremented'' :: Num a => [a] -> Maybe a
first_incremented'' = fmap (+1) . first' --more on this if we have time!
first_incremented'' [3,2,1]
first_incremented'' []

4

Just 4

Nothing

Just 4

Nothing

# Functional Data Structures

In [14]:
-- List
data List a = EmptyList | List a (List a)

listContains :: Eq a => List a -> a -> Bool
listContains EmptyList _ = False
listContains (List val rest) x = (val == x) || listContains rest x

-- Tree
data Tree a = EmptyTree | Tree (Tree a) a (Tree a)

treeContains :: Eq a => Tree a -> a -> Bool
treeContains EmptyTree _ = False
treeContains (Tree left val right) x = (val == x) || treeContains left x || treeContains right x

# Haskell Syntax (For reference!)

## Basic expressions

In [15]:
-- this is a comment

-- Basic types:
1   -- Integer
1.0 -- Double
True -- Boolean
'a' -- Char
"abc" -- String
(1,'a',"Alpha") -- Tuple
[1,2,3] -- List

-- Lists
1:[2,3] -- [1,2,3]

-- Pattern matching
(a,b) = (1,2) -- a and b are 1 and 2, respectively
(x:xs) = [1,2,3] -- x is 1, xs is [2,3]

-- Function application
negate 1 -- -1
not True -- False
gcd 15 20 -- 5
1 + 2 -- 3. Operators are just infix functions!
(+) 1 2 -- Prefix version

Line 1: Evaluate
Found:
not True
Why not:
False

1

1.0

True

'a'

...

-1

False

5

3

3

## Definitions and Types

In [16]:
-- Definitions
someVal = 3 -- Constant value
increment x = x + 1 -- Function taking one argument
add x y = x + y -- Function taking multiple arguments

-- Multiple definitions
sum' (x:xs) = x + sum' xs -- Recursively sum up a list
sum' [] = 0 -- Base case for empty list
 
-- Specifying types
someVal :: Int -- SomeVal is an Int
3 :: Float -- Here we want to make sure that 3 is read as a Float

decrement :: Int -> Int -- decrement is a function which takes an Int and returns an Int
decrement n = n - 1

multiply :: Int -> Int -> Int -- multiply takes 2 Ints and returns an Int
multiply x y = x * y

-- another view: multiply takes an Int and returns a function from Int to Int
triple :: Int -> Int
triple = multiply 3

-- Generic functions:
id' :: a -> a -- id' is a function which works for any type `a`. It takes an `a` and returns an `a`
id' x = x -- id' pretty much has to be defined this way, given its type signature

-- Constrained Generics
increment' :: Num a => a -> a -- A more generic signature for increment: will work for any Numeric type
increment' x = x + 1 -- Same function as before here

3.0

# Putting all that syntax together

In [17]:
map' :: (a -> b) -> [a] -> [b] -- map' takes a function from `a` to `b` and a list of `a`s, and returns a list of `b`s
map' f (x:xs) = f x : map' f xs -- apply `f` to the first element, and then to the rest
map' _ [] = [] -- mapping over an empty list just returns the empty list.
               -- We don't care what `f` is here, so we write "_" as a placeholder (same as in Python)

:t show
map' show [1,2,3,4]

show :: forall a. Show a => a -> String

["1","2","3","4"]